In [1]:
from fastcore.all import *
from fastai.vision.widgets import *
from fastai.vision.all import *

import os
from os.path import join

import PIL
import numpy as np

import matplotlib.pyplot as plt

import torch.nn
import torch

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
path

Path('C:/Users/0rc44/.fastai/data/mnist_sample')

In [3]:
train_path = path/'train'
test_path = path/'test'

In [4]:
labels = list()
data = list()
for root,dirs,files in os.walk(path):
    for file in files:
        if file.endswith(".png"):
            img = PIL.Image.open(join(root,file))
            data.append(np.asarray(img).flatten())
            labels.append(parent_label(join(root,file)))

# data[0]

labels = np.array(labels,dtype=int)
data = np.stack(data,axis=0)

In [5]:
data.shape

(14434, 784)

In [6]:
labels.shape

(14434,)

In [7]:
train_data = data / 255
test_data = (labels == 3).reshape(-1,1)

In [8]:
X = torch.tensor(train_data)
y = torch.tensor(test_data).long()

In [9]:
X

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [10]:
y

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]])

In [11]:
class Classifier(torch.nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(Classifier,self).__init__()
        self.linear = torch.nn.Linear(in_features=n_inputs,out_features=n_outputs)
        
    def forward(self,x):
        return self.linear(x.float()).sigmoid()

In [12]:
torch_model = Classifier(n_inputs = data.shape[-1],n_outputs = 1)

In [13]:
((torch_model(X) > 0.5) == y).float().mean()

tensor(0.5200)

In [14]:
epochs = 100

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(torch_model.parameters(),lr=0.01)


# training loop 
for epoch in range(epochs):
    y_pred = torch_model(X)
    loss = criterion(y_pred.flatten().float(),y.flatten().float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [15]:
((torch_model(X) > 0.5) == y).float().mean()

tensor(0.9675)